# Expected Points Model

### Purpose: 

The purpose of this Notebook is build a simple **expected points (xP) model** with a dataset extracted automatically from www.fbref.com. This example was developed to calculate the expected goals classification in this season in the LaLiga (Spain).

### Steps:


* Define a function to calculate the expected points (xP) value based on xG home team and xG away team using the Poisson distribution.

* Extract a list of matches from fbref.

* Iterate over each match to calculate xP_home_team & xP_away_team.

* Aggregate xP data for each team.

* Get the classification by points, expected points (xP) and difference between points & expedted points (xP). 

### References:

- https://www.pinnacle.com/en/betting-articles/Soccer/how-to-calculate-poisson-distribution/MD62MLXUMKMXZ6A8
- https://en.wikipedia.org/wiki/Poisson_distribution

### Author:

Alberto Rodríguez (@albertormweb)

In [1]:
import pandas as pd
from scipy.stats import poisson

Fbref url that cointains data for all matches in this season in LaLiga (Spain).

In [2]:
url_schedule = 'https://fbref.com/en/comps/12/schedule/La-Liga-Fixtures'

Function that calculates the number of points that gets a team in each match.

* Arguments: home_goals & away_goals.

* Return: list[home_points, away_points]

In [3]:
def calculate_points(home_goals, away_goals):

    home_points = away_points = 0
    point_list = []

    if home_goals > away_goals:
        home_points = 3
        away_points = 0
    if home_goals == away_goals:
        home_points = away_points = 1
    if home_goals < away_goals:
        home_points = 0
        away_points = 3
    
    point_list.append(home_points)
    point_list.append(away_points) 
    return point_list

Function that calculates the expected points value that gets a team in each match using the Poisson distribution.

* Arguments: xg_home_goals & xg_away_goals.

* Return: list[home_xp, away_xp]

In [4]:
def calculate_xp(xg_home, xg_away):

    prob_home_win = prob_draw = prob_away_win = value = 0
    xp_list = []

    for x in range(10): # simulate local goals
        for y in range (10): #simulate away goals
            value = round(((poisson.pmf(x,xg_home) * poisson.pmf(y,xg_away)) * 100),3) #poisson result
            if x > y:  #home win prob
                prob_home_win += value
            if x == y: #draw prob
                prob_draw+= value
            if x < y:  #away win prob
                prob_away_win += value
    
    xp_list.append(((prob_home_win/100) * 3) + (prob_draw/100).round(2)) #home_xp = prob_win * 3 + prob_draw
    xp_list.append(((prob_away_win/100) * 3) + (prob_draw/100).round(2)) #away_xp = prob_win * 3 + prob_draw
    return xp_list

Import Scores & Fixtures table from fbref url into a pandas dataframe.

In [5]:
df = pd.read_html(url_schedule)[0]
df.head(15)

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2019-08-16,21:00,Athletic Club,0.5,1–0,0.9,Barcelona,47693.0,San Mamés,Carlos del Cerro,Match Report,NaN
1,1.0,Sat,2019-08-17,17:00,Celta Vigo,1.0,1–3,1.5,Real Madrid,23566.0,Estadio de Balaídos,Javier Estrada,Match Report,NaN
2,1.0,Sat,2019-08-17,19:00,Valencia,2.2,1–1,1.3,Real Sociedad,41846.0,Estadio de Mestalla,Jesús Gil,Match Report,NaN
3,1.0,Sat,2019-08-17,20:00,Mallorca,1.6,2–1,0.7,Eibar,15127.0,Iberostar Estadi,Mario Melero,Match Report,NaN
4,1.0,Sat,2019-08-17,21:00,Leganés,1.1,0–1,0.2,Osasuna,10020.0,Estadio Municipal de Butarque,Alberola Rojas,Match Report,NaN
5,1.0,Sat,2019-08-17,21:00,Villarreal,1.5,4–4,2.0,Granada,14753.0,Estadio de la Cerámica,Adrián Cordero,Match Report,NaN
6,1.0,Sun,2019-08-18,17:00,Alavés,0.4,1–0,0.6,Levante,12029.0,Estadio de Mendizorroza,César Soto,Match Report,NaN
7,1.0,Sun,2019-08-18,19:00,Espanyol,0.2,0–2,1.3,Sevilla,19832.0,RCDE Stadium,Juan Martínez,Match Report,NaN
8,1.0,Sun,2019-08-18,21:00,Betis,1.0,1–2,0.7,Valladolid,51497.0,Estadio Benito Villamarín,Santiago Jaime,Match Report,NaN
9,1.0,Sun,2019-08-18,22:00,Atlético Madrid,1.8,1–0,0.4,Getafe,55099.0,Estadio Wanda Metropolitano,Guillermo Cuadra,Match Report,NaN


Clean dataframe: 

* Drop uninterested columns.
* Select only the matches that have been played.
* Obtain home & away goles from score column.
* Rename dataframe columns.

In [6]:
#Drop uninterested columns: day, date, time, attendance, venue, referee, notes
df.drop(['Day', 'Date','Time', 'Attendance', 'Venue', 'Referee', 'Notes'], axis=1, inplace=True)
df.head(10)

,Wk,Home,xG,Score,xG.1,Away,Match Report
0,1.0,Athletic Club,0.5,1–0,0.9,Barcelona,Match Report
1,1.0,Celta Vigo,1.0,1–3,1.5,Real Madrid,Match Report
2,1.0,Valencia,2.2,1–1,1.3,Real Sociedad,Match Report
3,1.0,Mallorca,1.6,2–1,0.7,Eibar,Match Report
4,1.0,Leganés,1.1,0–1,0.2,Osasuna,Match Report
5,1.0,Villarreal,1.5,4–4,2.0,Granada,Match Report
6,1.0,Alavés,0.4,1–0,0.6,Levante,Match Report
7,1.0,Espanyol,0.2,0–2,1.3,Sevilla,Match Report
8,1.0,Betis,1.0,1–2,0.7,Valladolid,Match Report
9,1.0,Atlético Madrid,1.8,1–0,0.4,Getafe,Match Report


In [7]:
#Filter matches that have been played
df = df[df['Match Report']=='Match Report']
df.drop(['Match Report'], axis=1, inplace=True)
df.head(10)

,Wk,Home,xG,Score,xG.1,Away
0,1.0,Athletic Club,0.5,1–0,0.9,Barcelona
1,1.0,Celta Vigo,1.0,1–3,1.5,Real Madrid
2,1.0,Valencia,2.2,1–1,1.3,Real Sociedad
3,1.0,Mallorca,1.6,2–1,0.7,Eibar
4,1.0,Leganés,1.1,0–1,0.2,Osasuna
5,1.0,Villarreal,1.5,4–4,2.0,Granada
6,1.0,Alavés,0.4,1–0,0.6,Levante
7,1.0,Espanyol,0.2,0–2,1.3,Sevilla
8,1.0,Betis,1.0,1–2,0.7,Valladolid
9,1.0,Atlético Madrid,1.8,1–0,0.4,Getafe


In [8]:
# Obtain home & away goles from score column
result = df['Score'].str.split('–', n = 1, expand = True)
df['home_goals'] = result[0]
df['away_goals'] = result[1]
df.drop(['Score'], axis=1, inplace=True)
df.head(10)

,Wk,Home,xG,xG.1,Away,home_goals,away_goals
0,1.0,Athletic Club,0.5,0.9,Barcelona,1,0
1,1.0,Celta Vigo,1.0,1.5,Real Madrid,1,3
2,1.0,Valencia,2.2,1.3,Real Sociedad,1,1
3,1.0,Mallorca,1.6,0.7,Eibar,2,1
4,1.0,Leganés,1.1,0.2,Osasuna,0,1
5,1.0,Villarreal,1.5,2.0,Granada,4,4
6,1.0,Alavés,0.4,0.6,Levante,1,0
7,1.0,Espanyol,0.2,1.3,Sevilla,0,2
8,1.0,Betis,1.0,0.7,Valladolid,1,2
9,1.0,Atlético Madrid,1.8,0.4,Getafe,1,0


In [9]:
#Rename dataframe columns.
df.columns = ['week', 'home', 'xg_home', 'xg_away', 'away', 'home_goals', 'away_goals']
df.head(10)

,week,home,xg_home,xg_away,away,home_goals,away_goals
0,1.0,Athletic Club,0.5,0.9,Barcelona,1,0
1,1.0,Celta Vigo,1.0,1.5,Real Madrid,1,3
2,1.0,Valencia,2.2,1.3,Real Sociedad,1,1
3,1.0,Mallorca,1.6,0.7,Eibar,2,1
4,1.0,Leganés,1.1,0.2,Osasuna,0,1
5,1.0,Villarreal,1.5,2.0,Granada,4,4
6,1.0,Alavés,0.4,0.6,Levante,1,0
7,1.0,Espanyol,0.2,1.3,Sevilla,0,2
8,1.0,Betis,1.0,0.7,Valladolid,1,2
9,1.0,Atlético Madrid,1.8,0.4,Getafe,1,0


Iterate over each match to calculate xP and points of each team and add a column dataframe with those data:
* xp_home
* xp_away
* points_home
* points_away

In [10]:
for index, row in df.iterrows(): 
    xp_list = point_list = []
    xp_list = calculate_xp(row['xg_home'],row['xg_away'])  
    point_list = calculate_points(row['home_goals'],row['away_goals'])
    df.at[index,'xp_home'] = xp_list[0]  #add home xP into a new column called xP_home
    df.at[index,'xp_away'] = xp_list[1]  #add away xP into a new column called xP_away
    df.at[index,'points_home'] = point_list[0]  #add home points into a new column called points_home
    df.at[index,'points_away'] = point_list[1]  #add away points into a new column called points_away
    
df.head(10)

,week,home,xg_home,xg_away,away,home_goals,away_goals,xp_home,xp_away,points_home,points_away
0,1.0,Athletic Club,0.5,0.9,Barcelona,1,0,0.95629,1.67155,3.0,0.0
1,1.0,Celta Vigo,1.0,1.5,Real Madrid,1,3,1.01648,1.72385,0.0,3.0
2,1.0,Valencia,2.2,1.3,Real Sociedad,1,1,1.93514,0.85898,1.0,1.0
3,1.0,Mallorca,1.6,0.7,Eibar,2,1,2.02048,0.73480,3.0,0.0
4,1.0,Leganés,1.1,0.2,Osasuna,0,1,2.13136,0.54097,0.0,3.0
5,1.0,Villarreal,1.5,2.0,Granada,4,4,1.09078,1.70047,1.0,1.0
6,1.0,Alavés,0.4,0.6,Levante,1,0,1.06597,1.46917,3.0,0.0
7,1.0,Espanyol,0.2,1.3,Sevilla,0,2,0.45764,2.26730,0.0,3.0
8,1.0,Betis,1.0,0.7,Valladolid,1,2,1.57821,1.07742,0.0,3.0
9,1.0,Atlético Madrid,1.8,0.4,Getafe,1,0,2.36637,0.43515,3.0,0.0


Calculate the total points of the home team into a new dataframe.

In [11]:
df_home= df[['home', 'xp_home', 'points_home']]
df_home = df_home.groupby(['home']).sum()
df_home

,xp_home,points_home
home,,
Alavés,21.92289,23.0
Athletic Club,22.74500,24.0
Atlético Madrid,27.62060,29.0
Barcelona,29.98048,40.0
Betis,22.05938,24.0
Celta Vigo,19.30006,17.0
Eibar,18.15940,19.0
Espanyol,15.12451,8.0
Getafe,21.33298,25.0


Calculate the total points of the away team into a new dataframe.

In [12]:
df_away= df[['away', 'xp_away', 'points_away']]
df_away = df_away.groupby(['away']).sum()
df_away

,xp_away,points_away
away,,
Alavés,11.47549,9.0
Athletic Club,12.65760,13.0
Atlético Madrid,19.22459,16.0
Barcelona,20.54652,18.0
Betis,13.25544,9.0
Celta Vigo,13.99831,9.0
Eibar,11.29619,8.0
Espanyol,17.03845,12.0
Getafe,17.68785,21.0


Calculate the final dataframe. This dataframe should have three columns for each team: 
total points, total expected points and difference between points and xP.

Steps:

* Inner join between df_home & df_away.
* Create a new column called points that cointains the result of the sum of home points and away points.
* Create a new column called xp that cointains the result of the sum home xp and away xp.
* Create a new column diff that cointains the result of the difference between points and xp.
* Select only this new three columns.

In [13]:
df_result = pd.concat([df_home, df_away], axis=1, join='inner')
df_result = df_result.round(2)
df_result['points'] = df_result['points_home'] + df_result['points_away'] 
df_result['xp'] = df_result['xp_home'] + df_result['xp_away']
df_result['diff'] = df_result['points'] - df_result['xp']
df_result = df_result[['points','xp', 'diff']]
df_result

,points,xp,diff
Alavés,32.0,33.40,-1.40
Athletic Club,37.0,35.40,1.60
Atlético Madrid,45.0,46.84,-1.84
Barcelona,58.0,50.53,7.47
Betis,33.0,35.32,-2.32
Celta Vigo,26.0,33.30,-7.30
Eibar,27.0,29.46,-2.46
Espanyol,20.0,32.16,-12.16
Getafe,46.0,39.02,6.98
Granada,38.0,36.52,1.48


Sort df_result by points to get the real classification.

In [14]:
df_result.sort_values(by=['points'], ascending=False)

,points,xp,diff
Barcelona,58.0,50.53,7.47
Real Madrid,56.0,52.20,3.80
Sevilla,47.0,44.12,2.88
Real Sociedad,46.0,41.13,4.87
Getafe,46.0,39.02,6.98
Atlético Madrid,45.0,46.84,-1.84
Valencia,42.0,31.12,10.88
Villarreal,38.0,42.39,-4.39
Granada,38.0,36.52,1.48
Athletic Club,37.0,35.40,1.60


Sort df_result by expected points to get the xP classification.

In [15]:
df_result.sort_values(by=['xp'], ascending=False)

,points,xp,diff
Real Madrid,56.0,52.20,3.80
Barcelona,58.0,50.53,7.47
Atlético Madrid,45.0,46.84,-1.84
Sevilla,47.0,44.12,2.88
Villarreal,38.0,42.39,-4.39
Real Sociedad,46.0,41.13,4.87
Getafe,46.0,39.02,6.98
Osasuna,34.0,36.64,-2.64
Granada,38.0,36.52,1.48
Athletic Club,37.0,35.40,1.60


Sort df_result by difference between points and xp to get this classification.

In [16]:
df_result.sort_values(by=['diff'])

,points,xp,diff
Espanyol,20.0,32.16,-12.16
Leganés,23.0,34.42,-11.42
Celta Vigo,26.0,33.30,-7.30
Villarreal,38.0,42.39,-4.39
Mallorca,25.0,28.77,-3.77
Osasuna,34.0,36.64,-2.64
Eibar,27.0,29.46,-2.46
Betis,33.0,35.32,-2.32
Atlético Madrid,45.0,46.84,-1.84
Alavés,32.0,33.40,-1.40


-----------------------------------------------------------------------------

Alberto Rodríguez (@albertormweb) 
